In [ ]:
import torch
from torch import nn
import os
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def walkdir(dir_path):
    for dirpath,dirnames,filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} files in {dirpath}")
walkdir("../dataset/")

Hello world
